In [1]:
!pip install tokenizers==0.5.2

     |████████████████████████████████| 3.7MB 89kB/s 


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 163kB 41.8MB/s 
     |████████████████████████████████| 573kB 34.7MB/s 
     |████████████████████████████████| 204kB 53.4MB/s 
     |████████████████████████████████| 1.0MB 43.5MB/s 
     |████████████████████████████████| 890kB 61.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=3970f438130e5701117f65553048ae8f15f7ad6a27b622349b3117f4f094fc36
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=1988cfcb6f02c9989babdd752e1cbccf9755a295de1e15c1c28e08628508c4c5
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses


In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [0]:
import os
import re
import pandas as pd
import numpy as np

In [0]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value) # Python
    import numpy as np
    np.random.seed(seed_value) # cpu vars
    import torch
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [0]:
random_seed(42)

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Test.csv')
sub = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/SampleSubmission.csv')

In [15]:
train['label'][~train['label'].isin([0, -1, 1])] = -1
train = train.dropna()
test['safe_text'] = test['safe_text'].fillna('xxxxxx')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
train['label'] = train['label'] * train['agreement']
train = train[['safe_text','label']]
test = test[['safe_text']]

In [0]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error

In [18]:
y_pred_total = []

for i in range(5): 
    
    X_train, X_cv =  train_test_split(train, test_size=0.20, random_state=i, stratify=train['label'])
    
    model = ClassificationModel('roberta', 'roberta-base', num_labels=1, use_cuda=True, 
                            args={'fp16': False,
                                  'learning_rate': 4e-5,
                                  'regression': True, 
                                  'overwrite_output_dir': True, 
                                  'num_train_epochs': 3,
                                  'manual_seed': i
                                  }
                            )
    
    model.train_model(X_train)
    scores, model_outputs, wrong_predictions = model.eval_model(X_cv)
    print('RMSE:', np.sqrt(mean_squared_error(X_cv['label'], model_outputs)))
    
    predictions, raw_output = model.predict(test['safe_text'])
    y_pred_total.append(predictions)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.271978

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.046905


Running loss: 0.028552


...

Running loss: 0.332425


Running loss: 0.052216


Running loss: 0.544034



/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



RMSE: 0.473702391304223


In [25]:
np.mean(y_pred_total, 0)

array([0.44925046, 0.56846935, 0.0024838 , ..., 0.31937104, 0.91028845,
       0.35317436], dtype=float32)

In [28]:
y_pred_total

[array([0.7050068 , 0.5999411 , 0.00846733, ..., 0.28790167, 0.9507355 ,
        0.16304164], dtype=float32),
 array([-0.0024948 ,  0.6295264 ,  0.0424838 , ...,  0.70276535,
         0.85173386,  0.6667481 ], dtype=float32),
 array([0.5486028 , 0.41290286, 0.01159108, ..., 0.0954884 , 0.9871822 ,
        0.30403602], dtype=float32),
 array([ 0.8707474 ,  0.56845033, -0.00577811, ...,  0.1928134 ,
         0.96104753,  0.5317027 ], dtype=float32),
 array([ 0.12439015,  0.6315261 , -0.04434508, ...,  0.3178864 ,
         0.8007431 ,  0.10034326], dtype=float32)]

In [0]:
y_pred = np.mean(y_pred_total, 0)

In [30]:
sub['label'] = y_pred
sub.head()

,tweet_id,label
0,00BHHHP1,0.449250
1,00UNMD0E,0.568469
2,01AXPTJF,0.002484
3,01HOEQJW,0.893113
4,01JUKMAO,0.071750


In [31]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(sub)